In [1]:
# Dependencies
import pandas as pd
import json
import csv

In [5]:
#Filepaths
RE = "Resources/real_estate.csv"
CENSUS = "Resources/census.CSV"

#File paths to DF
#REM to DF
df_RE=pd.read_csv(RE, encoding="ISO-8859-1", dtype=str)
df_CEN=pd.read_csv(CENSUS, encoding="ISO-8859-1", dtype=str)

#View DF's
# df_RE
# df_CEN

#View Dtypes
# print(df_RE.dtypes)
# print(df_CEN.dtypes)

In [15]:
#Merge DFs
merged_df=pd.merge(df_RE,df_CEN)
#merged_df

In [16]:
#Change dtypes to their corresponding types
#merged_df.dtypes
df=merged_df.astype({'Average RE Prices': 'float64','Income per capita': 'float64','Median Age Male': 'float64','Median Age Male.1': 'float64',"Bachelor's degree > 25": 'float64',"Public transportation": 'float64'})
df.dtypes

zipcode                    object
State                      object
City                       object
Metro                      object
CountyName                 object
Average RE Prices         float64
Income per capita         float64
Median Age Male           float64
Median Age Male.1         float64
Bachelor's degree > 25    float64
Median Home Value          object
Public transportation     float64
dtype: object

In [18]:
#Clean df columns for final
drop_df=df.drop(columns=['Metro','Median Home Value'])
final_df = drop_df.rename(columns={'Median Age Male.1':"Median Age Female"})
final_df

,zipcode,State,City,CountyName,Average RE Prices,Income per capita,Median Age Male,Median Age Female,Bachelor's degree > 25,Public transportation
0,00602,AL,Auburn,Lee County,114105.2500,9277.0,41.9,42.8,0.134690,0.002796
1,00606,MO,Neosho,Newton County,59377.7500,5943.0,43.2,43.4,0.068866,0.000000
2,00610,KY,Anasco,Pike County,108219.5833,10220.0,40.4,44.2,0.130349,0.002741
3,00617,AL,Imbery,Coffee County,101022.0000,9122.0,39.2,41.0,0.157224,0.005154
4,00693,PA,Greensburg,Westmoreland County,330761.5000,10536.0,39.2,41.9,0.148064,0.006750
...,...,...,...,...,...,...,...,...,...,...
29733,99827,AK,Haines,Haines Borough,211487.0000,33387.0,45.2,45.8,0.229525,0.002308
29734,99833,AK,Petersburg,Petersburg Borough,254642.9167,33227.0,39.3,42.7,0.219672,0.009211
29735,99835,AK,Sitka,Sitka Borough,402391.3333,38423.0,37.2,40.5,0.204494,0.010178
29736,99901,AK,Ketchikan,Ketchikan Gateway Borough,294448.8333,34886.0,39.4,39.9,0.170142,0.035648


In [19]:
#Create CSV
final_df.to_csv("Master.csv",index=False)

In [ ]:
#Check for errors
# df_CHECK=pd.read_csv("Master.csv", encoding="ISO-8859-1", converters={'zipcode': lambda x: str(x)})
# df_CHECK.set_index('zipcode')